# Systemy uczące się - Zad. dom. 1: Minimalizacja ryzyka empirycznego

Celem zadania jest zaimplementowanie własnego drzewa decyzyjnego wykorzystującego idee minimalizacji ryzyka empirycznego.

### Autor rozwiązania

Uzupełnij poniższe informacje umieszczając swoje imię i nazwisko oraz numer indeksu:


In [1]:
NAME = "Mikołaj Nowak"
ID = "151813"

## Twoja implementacja

Twoim celem jest uzupełnić poniższą klasę `TreeNode` tak by po wywołaniu `TreeNode.fit` tworzone było drzewo decyzyjne minimalizujące ryzyko empiryczne. Drzewo powinno wspierać problem klasyfikacji wieloklasowej (jak w przykładzie poniżej). Zaimplementowany algorytm nie musi (ale może) być analogiczny do zaprezentowanego na zajęciach algorytmu dla klasyfikacji. Wszelkie przejawy inwencji twórczej wskazane. **Pozostaw komenatrze w kodzie, które wyjaśniają Twoje rozwiązanie.**

Schemat oceniania:

- wynik na zbiorze Iris (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 10%: +40%,
- wynik na ukrytym zbiorze testowym 1 (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 15%: +30%,
- wynik na ukrytym zbiorze testowym 2 (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 5%: +30%.

Niedozwolone jest korzystanie z zewnętrznych bibliotek do tworzenia drzewa decyzyjnego (np. scikit-learn).
Możesz jedynie korzystać z biblioteki numpy.

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod), o ile będzie się w nim na koniec znajdowała kompletna implementacja klasy `TreeNode` w jednej komórce.


In [2]:
import numpy as np
from collections import Counter

class TreeNode:
	def __init__(self):
		self.left: TreeNode | None = None  # Wierzchołek znajdujący się po lewej stronie
		self.right: TreeNode | None = None  # Wierzchołek znajdujący się po prawej stronie
		self.feature: int | None = None  # Indeks cechy, według której dokonano podziału
		self.threshold: float | None = None  # Wartość progowa podziału
		self.answer: int | None = None  # Klasa, jeśli wierzchołek jest liściem

	def fit(self, data: np.ndarray, target: np.ndarray) -> None:
		"""
		Args:
			data (np.ndarray): macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech
			target (np.ndarray): wektor klas o długości n, gdzie n to liczba przykładów
		"""
		# Nie powinno się wydarzyć, ale lepiej zabezpieczyć
		if len(target) == 0:
			self.answer = 0
			return
		bestCut = [0.0, None, None] # zysk, cecha, threshold
		target_counts = Counter(target)  # Tworzy słownik {wartość: liczba_wystąpień}
		most_common_element, count = Counter(target).most_common(1)[0]
		accuracy = count/len(target)
		# Unikaj overfittingu poprzez niedzielenie małych zbiorów
		# Tak samo jeśli dokładność początkowa jest wystarczająco duża to nie dziel już więcej
		if(len(data) < 3 or accuracy > 0.9):
			self.answer = most_common_element
			return
		num_features = data.shape[1]  # Liczba cech (kolumn)
		for feature in range(num_features):
			sorted_indices = data[:, feature].argsort()  # Indeksy sortowania dla danej cechy
			sorted_data = data[sorted_indices]  # Posortowane wiersze macierzy cech
			sorted_target = target[sorted_indices]  # Odpowiadający im posortowany wektor klas
			for i in range(len(sorted_target) - 1):
				if sorted_target[i] != sorted_target[i + 1]:  # Znaleziono potencjalny próg podziału
					threshold = (sorted_data[i, feature] + sorted_data[i + 1, feature]) / 2
                    # Tworzenie podzbiorów
					mask_left = data[:, feature] < threshold
					mask_right = data[:, feature] >= threshold
					data_left = data[mask_left]
					data_right = data[mask_right]
					target_left = target[mask_left]
					target_right = target[mask_right]
					divided_counts_left = Counter(target_left)
					divided_counts_right = Counter(target_right)
					# 1 podział: Wszystko po lewo to jedna klasa
					# Policz accuracy początkowe bez podziału
					initial_accuracy = target_counts[sorted_target[i]] / len(data)
					# Policz accuracy po podziale
					cardinality_left = (len(data_left)/len(data))
					accuracy_left = (divided_counts_left[sorted_target[i]] / len(data_left)) if len(data_left) > 0 else 0
					cardinality_right = (len(data_right)/len(data))
					accuracy_right = 1 - (divided_counts_right[sorted_target[i]] / len(data_right)) if len(data_right) > 0 else 1
					new_accuracy = cardinality_left*accuracy_left + cardinality_right*accuracy_right
					if(new_accuracy - initial_accuracy > bestCut[0]):
						bestCut = [new_accuracy - initial_accuracy, feature, threshold]
					# 2 podział: Wszystko po prawo to jedna klasa
					# Policz accuracy początkowe bez podziału
					initial_accuracy = target_counts[sorted_target[i+1]] / len(data)
					# Policz accuracy po podziale
					cardinality_left = (len(data_left)/len(data))
					accuracy_left = 1 - (divided_counts_left[sorted_target[i+1]] / len(data_left)) if len(data_left) > 0 else 1
					cardinality_right = (len(data_right)/len(data))
					accuracy_right = (divided_counts_right[sorted_target[i+1]] / len(data_right)) if len(data_right) > 0 else 0
					new_accuracy = cardinality_left*accuracy_left + cardinality_right*accuracy_right
					if(new_accuracy - initial_accuracy > bestCut[0]):
						bestCut = [new_accuracy - initial_accuracy, feature, threshold]
		if(bestCut[0] > 0.0):
			self.left = TreeNode()
			self.right = TreeNode()
			self.feature = bestCut[1]
			self.threshold = bestCut[2]
			feature = bestCut[1]
			threshold = bestCut[2]
			mask_left = data[:, feature] < threshold
			mask_right = data[:, feature] >= threshold
			data_left = data[mask_left]
			data_right = data[mask_right]
			target_left = target[mask_left]
			target_right = target[mask_right]
			self.left.fit(data_left, target_left)
			self.right.fit(data_right, target_right)	
		else:
			self.answer = most_common_element
	
	def predict(self, data: np.ndarray) -> np.ndarray:
		"""
    	Args:
        	data (np.ndarray): macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech
    	Returns:
        	np.ndarray: wektor przewidzianych klas o długości n, gdzie n to liczba przykładów
    	"""
		y_pred = np.zeros(data.shape[0], dtype=int)  # Tworzymy tablicę na wyniki
		
		for i, row in enumerate(data):  # Iterujemy przez każdy wiersz (przykład)
			node = self  # Zaczynamy od korzenia drzewa
			while node.answer is None:  # Dopóki nie dotrzemy do liścia
				if row[node.feature] < node.threshold:  # Sprawdzamy warunek podziału
					node = node.left
				else:
					node = node.right
			y_pred[i] = node.answer  # Przypisujemy klasę liścia do predykcji
		return y_pred

## Przykład trenowanie i testowania drzewa

Później znajduje się przykład trenowania i testowania drzewa na zbiorze danych `iris`, który zawierający 150 próbek irysów, z czego każda próbka zawiera 4 atrybuty: długość i szerokość płatków oraz długość i szerokość działki kielicha. Każda próbka należy do jednej z trzech klas: `setosa`, `versicolor` lub `virginica`, które są zakodowane jak int.

Możesz go wykorzystać do testowania swojej implementacji. Możesz też zaimplementować własne testy lub użyć innych zbiorów danych, np. innych [zbiorów danych z scikit-learn](https://scikit-learn.org/stable/datasets/toy_dataset.html#toy-datasets).


In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=2024)

tree_model = TreeNode()
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9
